# Train

Enfim chegamos na etapa de treinamento do modelo, para esse problema vamos utilizar um modelo de boosting. Esse tipo de modelo foi escolhido devido as características encontradas no problema:  
- Problema desbalanceado, modelos de boosting conseguem se adequar bem a esse tipo de problema devido a classificação errada incorporar um peso entre as iterações.  
- É um modelo robusto e de fácil treinamento.  
- Por mais que seja um modelo "caixa-preta", é possível trazer explicabilidade através de bibliotecas como shap e lime. 

In [1]:
import pandas as pd
import lightgbm as lgbm
import yaml
import pickle
import sys
from sklearn.metrics import roc_auc_score

sys.path.append('../')

# Data

In [2]:
df_train = pd.read_parquet('../data/encoded/fraud_dataset_v2_train.parquet.gzip')
df_valid = pd.read_parquet('../data/encoded/fraud_dataset_v2_valid.parquet.gzip')
df_test = pd.read_parquet('../data/encoded/fraud_dataset_v2_test.parquet.gzip')

In [3]:
params = yaml.safe_load(open('../src/model/config/hp_hparams.yml', 'r'))
selector = pickle.load(open('../model/encoders/selector.pkl', 'rb'))

# Train

In [4]:
model = lgbm.LGBMClassifier(
    verbose = 0,
    random_state = 777,
    importance_type = 'gain',
    # Early stopping changed to 2, to reduce overfitting
    early_stopping_rounds = 2,
    objective= 'binary',
    boosting_type= 'gbdt',
    n_jobs=-1,
    **params
)

In [5]:
model = model.fit(selector.transform(df_train),
                  df_train['fraude'],
                  eval_set=[(
                    selector.transform(df_valid),
                    df_valid['fraude']
                  )], 
              eval_metric='auc'
              )

[LightGBM] [Warning] early_stopping_round is set=2, early_stopping_rounds=2 will be ignored. Current value: early_stopping_round=2
[LightGBM] [Warning] early_stopping_round is set=2, early_stopping_rounds=2 will be ignored. Current value: early_stopping_round=2


In [6]:
y_pred_train = model.predict_proba(selector.transform(df_train))[:,1]
y_pred_valid = model.predict_proba(selector.transform(df_valid))[:,1]
y_pred_test = model.predict_proba(selector.transform(df_test))[:,1]

In [7]:
print(f"ROCAUC Train: {roc_auc_score(df_train['fraude'], y_pred_train)}")
print(f"ROCAUC Valid: {roc_auc_score(df_valid['fraude'], y_pred_valid)}")
print(f"ROCAUC Test: {roc_auc_score(df_test['fraude'], y_pred_test)}\n")

print(f"ROCAUC@1% Train: {roc_auc_score(df_train['fraude'], y_pred_train, max_fpr=0.01)}")
print(f"ROCAUC@1% Valid: {roc_auc_score(df_valid['fraude'], y_pred_valid, max_fpr=0.01)}")
print(f"ROCAUC@1% Test: {roc_auc_score(df_test['fraude'], y_pred_test, max_fpr=0.01)}")

ROCAUC Train: 0.8858946521894506
ROCAUC Valid: 0.8789279997746019
ROCAUC Test: 0.8760424727223224

ROCAUC@1% Train: 0.5915467034896126
ROCAUC@1% Valid: 0.591944544138385
ROCAUC@1% Test: 0.5769488154142223


A decisão de redução do early stopping não foi tomada em cima do dataset de teste, apenas em cima do dataset de treino e validação.  
Considerando o dataset de treino e validação, temos uma queda de performance entre os dois, quando realizado um teste com early stopping = 5 a queda era ainda maior, por isso a redução de 5 para 2.  
Quando olhamos conta o dataset de teste a queda é ainda maior, vamos seguir com a analise dos resultados para entender se essa queda é constante ao longo do tempo, caso seja o ideial é voltar para etapas anteriores ao treinamento (algo normal no ciclo de modelagem do CRISP-DM).  